In [1]:
### Make calculations with results from QGIS
import geopandas as gpd
import pandas as pd
import requests
import pyproj
from shapely.geometry import shape
from shapely.ops import transform
from tqdm.notebook import tqdm
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
import numpy as np
from collections import Counter
from prettytable import PrettyTable
from datetime import datetime

In [2]:
# Read the shapefile
gdf = gpd.read_file('data/sms_in_15min_4326.shp')
# Rename some columns
new_column_names = {
    "hws": "Einwohner",
    "NAME_1": "Bundesland",
    "NAME_2": "Bezirk",
    "field_1": "sup_lat",
    "field_2": "sup_long"
}

# Create a new GeoPandas DataFrame with the renamed columns and the rest unchanged
gdf = gdf.rename(columns=new_column_names)
gdf

,fid,OBJECTID_1,ID,NAME,x,OBJECTID,cellcode,cellcode_X,cellcode_Y,Einwohner,...,Shape_Leng,Shape_Area,Bundesland,Bezirk,n_sm_in_15,sup_lat,sup_long,lat_epsg30,long_epsg3,geometry
0,1.0,230172.0,250mN281275E480375,CRS3035RES250mN2812750E4803750,0.0,200611,250mN281275E480375,-55.166667,48.25,1,...,1000.0,62500.0,Wien,Wien,3.0,48.226186,16.505093,2.811583e+06,4.803829e+06,"POLYGON ((16.52598 48.23661, 16.52456 48.23279..."
1,2.0,230171.0,250mN281250E480375,CRS3035RES250mN2812500E4803750,0.0,200265,250mN281250E480375,-55.166667,48.25,3,...,1000.0,62500.0,Wien,Wien,5.0,48.226186,16.505093,2.811583e+06,4.803829e+06,"POLYGON ((16.52569 48.23438, 16.52427 48.23055..."
2,3.0,230170.0,250mN281275E480350,CRS3035RES250mN2812750E4803500,0.0,200610,250mN281275E480350,-55.166667,48.25,181,...,1000.0,62500.0,Wien,Wien,4.0,48.226186,16.505093,2.811583e+06,4.803829e+06,"POLYGON ((16.52262 48.23681, 16.52120 48.23299..."
3,4.0,230165.0,250mN281275E480325,CRS3035RES250mN2812750E4803250,0.0,200609,250mN281275E480325,-55.166667,48.25,274,...,1000.0,62500.0,Wien,Wien,4.0,48.226186,16.505093,2.811583e+06,4.803829e+06,"POLYGON ((16.51927 48.23701, 16.51785 48.23319..."
4,5.0,228905.0,250mN281150E480375,CRS3035RES250mN2811500E4803750,0.0,198929,250mN281150E480375,-55.166667,48.25,416,...,1000.0,62500.0,Wien,Wien,5.0,48.226186,16.505093,2.811583e+06,4.803829e+06,"POLYGON ((16.52453 48.22542, 16.52311 48.22160..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440031,440032.0,225033.0,250mN275300E480775,CRS3035RES250mN2753000E4807750,0.0,124948,250mN275300E480775,-55.250000,48.25,7,...,1000.0,62500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"POLYGON ((16.51074 47.69818, 16.50934 47.69436..."
440032,440033.0,234178.0,250mN286675E476625,CRS3035RES250mN2866750E4766250,0.0,234420,250mN286675E476625,-55.166667,48.25,2,...,1000.0,62500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"POLYGON ((16.08124 48.74919, 16.07984 48.74536..."
440033,440034.0,234179.0,250mN286725E476625,CRS3035RES250mN2867250E4766250,0.0,234504,250mN286725E476625,-55.166667,48.25,2,...,1000.0,62500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"POLYGON ((16.08179 48.75367, 16.08039 48.74984..."
440034,440035.0,234180.0,250mN286700E476650,CRS3035RES250mN2867000E4766500,0.0,234463,250mN286700E476650,-55.166667,48.25,6,...,1000.0,62500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"POLYGON ((16.08490 48.75124, 16.08350 48.74742..."


In [3]:
# Create a custom function to switch the coordinates of a Polygon
def switch_coordinates(polygon):
    exterior = list(polygon.exterior.coords)
    interior = [list(interior.coords) for interior in polygon.interiors]
    switched_exterior = [(y, x) for x, y in exterior]  # Switch the coordinates
    switched_interior = [[(y, x) for x, y in ring] for ring in interior]  # Switch the coordinates

    return Polygon(switched_exterior, switched_interior)

# Apply the custom function to the "geometry" column
gdf['geometry_new'] = gdf['geometry'].apply(switch_coordinates)

# Now, the coordinates of each polygon in the DataFrame have been switched

In [4]:
# Calculate centroid of the raster
gdf['centroid'] = gdf['geometry_new'].centroid

C:\Users\lukas\AppData\Local\Temp\ipykernel_18488\2571640796.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry_new'].centroid


In [5]:
gdf_final = gdf.loc[:, ['OBJECTID', 'Einwohner', 'Bundesland', 'Bezirk', 'n_sm_in_15', 'sup_lat', 'sup_long', 'centroid']]
gdf_final = gdf_final.sort_values(by='OBJECTID')
gdf_final

,OBJECTID,Einwohner,Bundesland,Bezirk,n_sm_in_15,sup_lat,sup_long,centroid
300639,1,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41064 14.56177)
300645,2,3,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41500 14.56540)
300629,3,2,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41890 14.52667)
300635,4,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41780 14.55262)
300628,5,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.42114 14.52686)
...,...,...,...,...,...,...,...,...
315284,236867,4,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01098 15.03515)
315279,236868,1,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01368 15.02515)
315443,236869,4,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01199 15.06266)
440030,236870,2,NaN,NaN,0.0,NaN,NaN,POINT (49.01623 15.01856)


In [14]:
### OSRM APPROACH

typeOfTravel = "foot"
# OSRM server URL
osrm_url = "http://router.project-osrm.org/route/v1/"
osrm_url = f"{osrm_url}{typeOfTravel}/"

def calc_walking_distance(start_coords, end_coords, osrm_url):
    

    # Format coordinates as "longitude,latitude"
    start_coords_str = ",".join(map(str, start_coords[::-1]))
    end_coords_str = ",".join(map(str, end_coords[::-1]))

    # Build the request URL
    request_url = f"{osrm_url}{start_coords_str};{end_coords_str}?overview=full&alternatives=false"

    # Make the request to the OSRM API
    response = requests.get(request_url)

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract the distance in meters from the response
        distance = data["routes"][0]["distance"]

        # Convert the distance to kilometers and duration to minutes
        distance_km = distance / 1000
        duration_min = distance_km / 5 * 60

        return duration_min
    else:
        return None

In [16]:
anfang = datetime.now()
res = []

print('Anfang: ', anfang)
for i in tqdm(range(len(gdf_final)), total = len(gdf_final), desc = 'Calculating Walking Distances'):
    
    if (gdf_final['n_sm_in_15'].iloc[i] == 0):
        res.append(np.nan)
        continue
    
    res.append(calc_walking_distance((gdf_final['centroid'].iloc[i].x, gdf_final['centroid'].iloc[i].y), 
                                     (gdf_final['sup_lat'].iloc[i], gdf_final['sup_long'].iloc[i]),
                                    osrm_url))

        
ende = datetime.now()
print('Ende: ', ende)
print(ende-anfang)

Anfang:  2023-11-11 21:23:36.475676


Calculating Walking Distances:   0%|          | 0/440036 [00:00<?, ?it/s]

Ende:  2023-11-13 13:07:55.447628
1 day, 15:44:18.971952


In [26]:
gdf_final['walkdist_in_minutes'] = res
gdf_final['dist_km'] = gdf_final['walkdist_in_minutes'] / 60 *5
gdf_final

,OBJECTID,Einwohner,Bundesland,Bezirk,n_sm_in_15,sup_lat,sup_long,centroid,walkdist_in_minutes,dist_km
300639,1,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41064 14.56177),NaN,NaN
300645,2,3,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41500 14.56540),NaN,NaN
300629,3,2,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41890 14.52667),NaN,NaN
300635,4,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.41780 14.55262),NaN,NaN
300628,5,1,Kärnten,Völkermarkt,0.0,NaN,NaN,POINT (46.42114 14.52686),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
315284,236867,4,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01098 15.03515),NaN,NaN
315279,236868,1,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01368 15.02515),NaN,NaN
315443,236869,4,Niederösterreich,Gmünd,0.0,NaN,NaN,POINT (49.01199 15.06266),NaN,NaN
440030,236870,2,NaN,NaN,0.0,NaN,NaN,POINT (49.01623 15.01856),NaN,NaN


In [28]:
gdf_final.to_csv('data/gdf_mit_walkdist_in_min.csv',
                index=False, encoding='utf-8')